In [1]:
import os, sys

recsys_cf = os.environ.get("RECSYS_STORY")
sys.path.append(recsys_cf)
os.chdir(recsys_cf)

from lib.tools import *

### SPARK 

In [2]:
sp = spark(**SPARK_CONFIG)
hive = sp.sql

In [6]:
# %%time
# insight_table = load_table("sbx_t_team_cvm", 'ma_mmb_insight', hive).cache()

In [3]:
%%time
insight_table = load_table(SBX_TEAM_DIGITCAMP, "sbx_t_team_cvm_ma_mmb_insight_tmp", hive).cache()

CPU times: user 2.83 ms, sys: 3.45 ms, total: 6.28 ms
Wall time: 3.19 s


In [4]:
%%time
insight_table.count()

CPU times: user 77.1 ms, sys: 97.8 ms, total: 175 ms
Wall time: 8min 57s


38068405

In [ ]:
insight_table.show()

In [7]:
insight_table.columns

['INSIGHT_ID',
 'INN',
 'SOURCE_CD',
 'CREATION_DTTM',
 'INSIGHT_START_DT',
 'INSIGHT_END_DT',
 'SCORE_VAL',
 'INSIGHT_DESC',
 'ACTIVITY_TYPE_CD',
 'REASON',
 'SRC_ID',
 'CKR',
 'MS',
 'MKK',
 'SBBOL',
 'PRODUCT_ID',
 'GKM',
 'SAS_DYN_PAR_ID',
 'DYN_FLG',
 'sources_lv2_cd']

In [14]:
insight_table.select('INN',
                     'sources_lv2_cd',
                     f.to_date('CREATION_DTTM').alias('CREATE_DATE'),
                     'SCORE_VAL',
                     'PRODUCT_ID'
                    )\
             .groupBy("INN", "PRODUCT_ID", "CREATE_DATE").agg(f.collect_set('sources_lv2_cd').alias("source_list"),
                                                              f.mean('SCORE_VAL').alias("SCORE")).count()

36578700

In [ ]:
insight_table.select('INN',
                     'sources_lv2_cd',
                     f.to_date('CREATION_DTTM').alias('CREATE_DATE'),
                     'SCORE_VAL',
                     'PRODUCT_ID'
                    )\
             .groupBy("INN", "PRODUCT_ID", "CREATE_DATE").agg(f.collect_set('sources_lv2_cd').alias("source_list"),
                                                              f.mean('SCORE_VAL').alias("SCORE")).show()

cid sbbol inn

In [32]:
cid_sbbol = load_table(SBX_TEAM_DIGITCAMP, "GA_CID_SBBOL_INN" , hive).cache()

In [ ]:
cid_sbbol.show()

In [34]:
cid_sbbol.select(f.max('load_dt')).show()

,max(load_dt)
0,2021-05-14


In [35]:
%%time
cid_sbbol.count()

CPU times: user 2.98 ms, sys: 831 µs, total: 3.81 ms
Wall time: 1.83 s


497054980

In [ ]:
cid_sbbol.select('cu_inn', 'load_dt')\
         .groupBy('load_dt').agg(f.count('cu_inn'))\
         .orderBy(f.col('load_dt').desc())\
         .show(30)

In [ ]:
load_table(SBX_TEAM_DIGITCAMP, 'MA_CUSTOMER_COOKIE_MAP', hive).show()

In [36]:
cookie = load_table(SBX_TEAM_DIGITCAMP, 'MA_CUSTOMER_COOKIE_MAP', hive).cache()

In [37]:
cookie.select(f.max('SRC_UPDATE_DTTM')).show()

,max(SRC_UPDATE_DTTM)
0,2021-08-21


In [38]:
customer = load_table(SBX_TEAM_DIGITCAMP, 'GA_MA_CUSTOMER_SBBOL', hive).cache()

In [ ]:
customer.show()

In [50]:
################################## Resourses ##################################

#####################################################################
SBX_TEAM_DIGITCAMP = 'sbx_team_digitcamp'
##-----------------------------------------------------------------##
INTERNAL_PANHASH_COOKIES = "internal_panhash_cookies"
URLS_TO_PARSE_FOR_BUSINESS = "urls_to_parse_for_business"

ALL_COOKIES = "all_cookies"

MA_PRODUCT_DICT = 'ma_product_dict'
MA_CMDM_MA_DEAL = "ma_cmdm_ma_deal_new"
MA_MMB_OFFER_NONTOP = "ma_mmb_offer_nontop"

MW_ATB_SEGMEN_PROD_GROUP = "MW_ATB_SEGMEN_PROD_GROUP"

OFFER_PRIORITY = "offer_priority"

GA_CID_SBBOL_INN = "ga_cid_sbbol_inn"
GA_ALL_SCENARIOS_HIST = 'ga_all_scenarios_hist'
GA_ALL_SCENARIOS_STATS = "ga_all_scenarios_stats"
GA_SITE_ALL_PRODS = "GA_SITE_ALL_PRODS_SLICE"

TMP_VISIT_PART = 'tmp_visit_part'
TMP_TEST_VISIT_PART = 'TMP_VISIT_PART'
TMP_EXC_COPY_TABLE = "tmp_exc_copy_table"
TMP_SITE_ALL_PRODS_INTERMEDIATE = "TMP_SITE_ALL_PRODS_INTERMEDIATE"

UNIFIED_CUSTOMER = 'UNIFIED_CUSTOMER'
GA_MA_CUSTOMER_SBBOL = "GA_MA_CUSTOMER_SBBOL"
GA_MA_USER_PROFILE_SBBOL = "GA_MA_USER_PROFILE_SBBOL"
MA_CUSTOMER_COOKIE_MAP = "MA_CUSTOMER_COOKIE_MAP"
#####################################################################

#####################################################################
PROXY_GOOGLE_ANALYTICS ="prx_google_analytics_part_external_google_analytics"
SKLOD_EXTERNAL_GOOGLE_ANALYTICS = "sklod_external_google_analytics"
CAP_DDA_GOOGLE_ANALYTICS = "cap_external_google_dda_external_google_analytics"
##-----------------------------------------------------------------##
VISIT = 'visit'
#####################################################################

#####################################################################
SKLOD_EXTERNAL_CLICKSTREAM = "sklod_external_clickstream"
##-----------------------------------------------------------------##
CLICKSTREAM = 'clickstream'
#####################################################################

#####################################################################
SKLOD_NRT_SBBOL_CLICKSTREAM = "sklod_nrt_sbbol_clickstream"
##-----------------------------------------------------------------##
SBBOL_EVENTS = 'sbbol_events'
SBBOL_EVENTS_DELTA = 'sbbol_events_delta'
#####################################################################

################################## Iskra ##################################
ISKRA = 'iskra4'
##-----------------------------------------------------------------##
ISKRA_LOGIN = 'tech_iskra[iskra]'
ISKRA_PASS = 'Uthvfy123'
##-----------------------------------------------------------------##
ISKRA_BATCH_SIZE = 700000
#####################################################################

################################## ScenarioBase Data ##################################
SLICE_JSON_FILE = 'dates/slice.json'
SCENARIOS_JSON_FILE = 'dates/scenarios.json'
##-----------------------------------------------------------------##
TEST_MIN_CTL_LOADING_DEFAULT = 3709530
TEST_MAX_CTL_LOADING_DEFAULT = 3709946
#####################################################################

################################## SourcesUpdate Data ##################################
UNIFIED_CUSTOMER_JSON_FILE = 'dates/unified_customer.json'
SOURCES_JSON_FILE = 'dates/sources.json'
##-----------------------------------------------------------------##
REWRITABLE_SOURCES_LIST = [
    #'MA_MMB_OFFER_NONTOP' ,
    'UNIFIED_CUSTOMER', #error
    'MA_CUSTOMER_COOKIE_MAP',
    'GA_MA_CUSTOMER_SBBOL', #error
    'GA_MA_USER_PROFILE_SBBOL',
    'MA_PRODUCT_DICT',
]
##-----------------------------------------------------------------##
UPDATABLE_SOURCES_LIST = [
    #'OFFER_PRIORITY',
    #'MA_CMDM_MA_DEAL'
]

DATE_COLUMN_IN_UPDATABLE_SOURCES = {
    'OFFER_PRIORITY': "load_dttm",
    #'MA_CMDM_MA_DEAL': "create_dt"
}
##-----------------------------------------------------------------##
FREQUNCY_0F_UPDATES = {
    'UNIFIED_CUSTOMER' : 3,
    'MA_CUSTOMER_COOKIE_MAP' : 3,
    'GA_MA_CUSTOMER_SBBOL' : 3,
    'GA_MA_USER_PROFILE_SBBOL' : 3,
    'MA_MMB_OFFER_NONTOP' : 3,
    'MA_PRODUCT_DICT' : 3,
    'OFFER_PRIORITY' : 3,
    'MA_CMDM_MA_DEAL' : 3
}
#####################################################################

################################## ScenarioStats Data ##################################
EMAIL_LIST_FILE = "mail_settings/mail_list.txt"
TEST_EMAIL_LIST_FILE = "mail_settings/test_mail_list.txt"
EMAIL_SETTINGS_FILE = "mail_settings/mail_settings.txt"
##-----------------------------------------------------------------##
ALL_SCENARIOS = [
    'STORIES_01',
    'DEPOSIT_01',
    'CREDIT_01',
    'WTA_SITE_NBS',
    'PREMIUMCARD_02',
    'HELP_SBBOL',
    'LETTERSOFCREDIT_01',
    'CORPCARD_01',
    'PREMIUMCARDMASTER_02',
    'DIGITALCARD_02',
    'CORPCARD_02',
    'DIGITALCARD_01',
    'PREMIUMCARDMASTER_01',
    'BUSINESS_CARD_01',
    'SITEALLPRODAGG_01',
    'SHOP_01',
    'PREMIUMCARD_01',
    'ACQUIRING_01',
    'CASHORDER_01',
    'ENCASHMENT_01',
    'COLLINSUR_ONLINE',
    'COLLINSUR_OTHER',
    'COLLINSUR_ALREADY']
#####################################################################


In [48]:
import os, sys
dlg = os.environ.get("DLG_CLICKSTREAM")
sys.path.insert(0, dlg)
os.chdir(dlg)



In [47]:
from lib.tools import *

In [49]:
JS

'/home/shubochkin1-ei_ca-sbrf-ru/notebooks/DLG_clickstream'

In [51]:
class CID_SBBOL_INN_UPDATE:

    ######################### Init and close ##############################

    def __init__(self):
#         self.sing = tendo.singleton.SingleInstance()
        self.REFDATE = "2019-09-01"
        self.SCENARIO_ID = 'CIDSBBOLUSERID'
        self.script_name = "CID_SBBOL_USER_ID"
        self.currdate = datetime.strftime(datetime.today(), format='%Y-%m-%d')
        
        self.init_logger()
        log("# __init__ : begin", self.logger)

        self.load_scenarios_json()
        self.load_slice_json()
        self.load_sources_json()

        if self.slice_empty() or self.today_was_launch():
            pass
            #self.close()
            #sys.exit()

        self.start_spark()

        log("# __init__ : begin", self.logger)


    ############################## Spark ##############################


    @class_method_logger
    def start_spark(self):
#         self.sp = spark(schema=SBX_TEAM_DIGITCAMP,
#                         process_label=self.script_name,
#                         replication_num=2,
#                         kerberos_auth=True,
#                         numofcores=8,
#                         numofinstances=20)
        self.sp = sp
        self.hive = self.sp.sql


    ################################## Load json ##################################


    @class_method_logger
    def load_scenarios_json(self):
        with open(SCENARIOS_JSON_FILE, 'r') as f:
            self.scenarios_json = json.load(f)

        if self.SCENARIO_ID not in self.scenarios_json:
            self.scenarios_last_date = self.REFDATE
        else:
            self.scenarios_last_date = self.scenarios_json[self.SCENARIO_ID]

    @class_method_logger
    def load_slice_json(self):
        with open(SLICE_JSON_FILE, 'r') as f:
            self.slice_json = json.load(f)
        self.slice_count = self.slice_json['COUNT']
        self.slice_max_date = self.slice_json['MAX_DATE']


    @class_method_logger
    def load_sources_json(self):
        with open(SOURCES_JSON_FILE, 'r') as f:
            self.sources_json = json.load(f)
        self.last_launch = self.sources_json[self.script_name]


    ################################## Check dates ##################################

    @class_method_logger
    def today_was_launch(self):
        if self.last_launch == self.currdate:
            log("### check_currdate_eq_last_launch_date ERROR: current_date == last_launch ", self.logger)
            log("table CID_SBBOL_INN was updated early today", self.logger)
            return True
        return False


    @class_method_logger
    def slice_empty(self):
        if self.slice_count == 0:
            log("### check_slice_count ERROR: slice_count = 0 >>> nothing to save", self.logger)
            return True
        return False


    ################################## Run ##################################


    @exception_restart(num_of_attempts=3, delay_time_sec=10*60)
    @class_method_logger
    def run(self):
        tmp_visit_part = self.load_visit_part()

        cid_sbbol, cid_segmento = self.get_cid_sbbol_and_segmento(tmp_visit_part)

        # Получение ИНН из связки с REPLICATIONGUID

        profile = self.load_user_profile()
        cid_sbbol_replicguid = cid_sbbol.join(profile, on=['sbbolUserId'], how='inner')


        customer = self.load_ma_customer()
        cid_sbbol_inn = cid_sbbol_replicguid.join(customer, on=['REPLICATIONGUID'], how='inner')


        # Получение ИНН из cookie_map

        #cookie_map = self.load_cookie_map()
        #active_cookie = self.get_active_cookie(cookie_map)

        #cid_segmento_inn = cid_segmento.join(active_cookie, 
        #                                   on=(cid_segmento.commonSegmentoUID == active_cookie.CUST_COOKIE), 
        #                                  how="left_outer")

        # Join

        cid_sbbol_inn_segmento = cid_sbbol_inn.join(cid_segmento, on=['cid'], how='full_outer').distinct() \
                                                #.withColumn('inn_',f.coalesce('CU_INN','CUST_INN')) \
                                                #.drop('CU_INN','CUST_INN')\
                                                #.withColumnRenamed("inn_", "CU_INN")


        unified_customer = self.load_unified_customer()

        cid_sbbol_inn_segmento_crm = cid_sbbol_inn_segmento.join(unified_customer, 
                                                                on=(cid_sbbol_inn_segmento.CU_INN == unified_customer.inn), how='leftouter') \
                                                                .dropDuplicates(['cu_inn','sbbolUserId', 'commonSegmentoUID', 'crm_id'])

        
        cid_sbbol_inn_segmento_crm_dt = self.add_load_dt(cid_sbbol_inn_segmento_crm)
        final_df = self.create_final_df(cid_sbbol_inn_segmento_crm_dt)

#         self.save_final_table(final_df)
#         self.save_scenarios_json(tmp_visit_part)
#         self.save_sources_json()

    ################################## Logging ##################################


    def init_logger(self):
        self.print_log = True

        try: 
            os.mkdir("logs/" + self.currdate)
        except:
            pass

        logging.basicConfig(filename='logs/{}/{}.log'.format(self.currdate, self.script_name),
                            level=logging.INFO,
                            format='%(asctime)s %(message)s')
        self.logger = logging.getLogger(__name__)

        log("="*54 + " {} ".format(self.currdate) + "="*54, self.logger)


    ################################## Load ##################################


    @class_method_logger
    def load_table(self, schema, table):
        return load_table(schema, table, self.hive)


    @class_method_logger
    def load_visit_part(self):
        tmp_visit_part = self.load_table(SBX_TEAM_DIGITCAMP, TMP_VISIT_PART) \
                            .filter("timestamp(sessionDate) > timestamp('{}')".format(self.scenarios_last_date))

        return tmp_visit_part


    @class_method_logger
    def get_cid_sbbol_and_segmento(self, visit_part):
        cid_sbbol = visit_part.filter('sbbolUserId is not Null') \
                                    .dropDuplicates(['cid', 'sbbolUserId']) \
                                    .select('cid', 'sbbolUserId', 'CTL_LOADING')

        cid_segmento = visit_part.filter('commonSegmentoUID is not Null') \
                                        .select('cid','commonSegmentoUID') \
                                        .distinct()

        return cid_sbbol, cid_segmento


    @class_method_logger
    def load_user_profile(self):
        profile = self.load_table(SBX_TEAM_DIGITCAMP, GA_MA_USER_PROFILE_SBBOL) \
                        .select('CU_ID_SBBOL', 'REPLICATIONGUID', 'USER_ID', f.col('USER_GUID').alias('sbbolUserId')) \
                        .filter("(sbboluserid is not Null) and not (sbboluserid = 'Пользователь Интернет-Клиента')")

        return profile


    @class_method_logger
    def load_ma_customer(self):
        customer = self.load_table(SBX_TEAM_DIGITCAMP, GA_MA_CUSTOMER_SBBOL) \
                        .select('REPLICATIONGUID', 'CU_INN', 'CU_KPP', 'CU_OKPO', 'LOCKED')

        return customer


    @class_method_logger
    def load_cookie_map(self):
        cookie_map = self.load_table(SBX_TEAM_DIGITCAMP, MA_CUSTOMER_COOKIE_MAP) \
                            .select("CUST_INN", "CUST_COOKIE", "SRC_UPDATE_DTTM") \

        return cookie_map


    @class_method_logger
    def load_unified_customer(self):
        unified_customer = self.load_table(SBX_TEAM_DIGITCAMP, UNIFIED_CUSTOMER)  \
                                .select('inn', 'crm_id').distinct()

        return unified_customer


    ################################## Operations ##################################

    @class_method_logger
    def get_active_cookie(self, cookie):
        cookie_active = cookie.withColumn("days_diff", f.datediff(f.current_timestamp(), f.col("SRC_UPDATE_DTTM"))) \
                                .filter(f.col("days_diff") <= 40) \
                                .select("CUST_INN", "CUST_COOKIE").distinct()
        return cookie_active


    @class_method_logger
    def add_load_dt(self, df):
        strdate = datetime.strftime(datetime.now(), format='%Y.%d.%m')
        df_with_load_df = df.withColumn('load_dt', f.lit(datetime.strptime(strdate, '%Y.%d.%m')).cast(stypes.TimestampType()))
        return df_with_load_df



    ############################## Create final_df ##############################


    @class_method_logger
    def create_final_df(self, df):
        cols = [ 'cid',
                'replicationguid',
                'sbboluserid',
                'commonSegmentoUID',
                'cu_id_sbbol',
                'user_id',
                'cu_inn',
                'cu_kpp',
                'crm_id',
                'cu_okpo',
                'locked',
                'load_dt',
                'ctl_loading']

        final_df = df.select([col if col != 'ctl_loading' else f.col(col).cast(stypes.LongType()) for col in cols])

        return final_df
                        

    ############################## Save final table ##############################

    @class_method_logger
    def save_final_table(self, df):
        tmp_tbl = 'tmp_inn'
        df.registerTempTable(tmp_tbl)

        self.hive.setConf("hive.exec.dynamic.partition", "true")
        self.hive.setConf("hive.exec.dynamic.partition.mode", "nonstrict")

        self.hive.sql(
            """
                insert into table {schema}.{tbl}
                partition(ctl_loading)
                select {tmp_tbl}.* from {tmp_tbl}
                distribute by ctl_loading
            """.format(schema=SBX_TEAM_DIGITCAMP, tbl=GA_CID_SBBOL_INN, tmp_tbl=tmp_tbl))
        return None


    @class_method_logger
    def save_scenarios_json(self, tmp_visit_part):
        ctl_loading = tmp_visit_part.select(f.max('sessionDate')).collect()
        self.slice_max_date = ctl_loading[0]['max(sessionDate)']

        self.scenarios_json[self.SCENARIO_ID] = self.slice_max_date
        with open(SCENARIOS_JSON_FILE, 'w') as obj:
            json.dump(self.scenarios_json, obj, indent=4, sort_keys=True)


    @class_method_logger
    def save_sources_json(self):
        self.sources_json[self.script_name] = self.currdate
        with open(SOURCES_JSON_FILE, 'w') as f:
            json.dump(self.sources_json, f, indent=4, sort_keys=True)


    ############################## ContextManager ##############################


    def __enter__(self):
        return self


    @class_method_logger
    def close(self):
        try:
            self.sp.sc.stop()
        except Exception as ex:
            if "object has no attribute" in str(ex):
                log("### SparkContext was not started", self.logger)
            else:
                log("### Close exception: \n{}".format(ex), self.logger)
        finally:
            pass
#             del self.sing


    def __exit__(self, exc_type, exc_value, exc_tb):
        log("# __exit__ : begin", self.logger)
        if exc_type is not None:
            log("### Exit exception ERROR:\nexc_type:\n{}\nexc_value\n{}\nexc_tb\n{}"\
                .format(exc_type, exc_value, exc_tb), self.logger)
        self.close()
        log("# __exit__ : end", self.logger)
        log("="*120, self.logger) 

In [52]:
class_cid_inn = CID_SBBOL_INN_UPDATE()

In [53]:
tmp_visit_part = class_cid_inn.load_visit_part()

In [54]:
tmp_visit_part.count()

0

In [58]:
print(class_cid_inn.scenarios_last_date)

None


In [60]:
class_cid_inn.SCENARIO_ID

'CIDSBBOLUSERID'

In [61]:
class_cid_inn.scenarios_json

{'ACQUIRING_01': '2021-08-22',
 'BUSINESS_CARD_01': '2021-08-22',
 'CASHORDER_01': '2021-08-22',
 'CIDSBBOLUSERID': None,
 'COLLINSUR_ALREADY': '2021-08-22',
 'COLLINSUR_ONLINE': '2021-08-22',
 'COLLINSUR_OTHER': '2021-08-22',
 'CORPCARD_01': '2021-08-22',
 'CORPCARD_02': '2021-08-22',
 'CREDIT_01': '2021-08-22',
 'DEPOSIT_01': '2021-08-22',
 'DIGITALCARD_01': '2021-08-22',
 'DIGITALCARD_02': '2021-08-22',
 'ENCASHMENT_01': '2021-08-22',
 'HELP_SBBOL': '2021-08-22',
 'LETTERSOFCREDIT_01': '2021-08-22',
 'PREMIUMCARDMASTER_01': '2021-08-22',
 'PREMIUMCARDMASTER_02': '2021-08-22',
 'PREMIUMCARD_01': '2021-08-22',
 'PREMIUMCARD_02': '2021-08-22',
 'SCENARIO_TEST': '2020-09-18',
 'SHOP_01': '2021-08-22',
 'SITEALLPRODAGG_01': '2021-08-22',
 'SITEALLPRODS_01': '2021-08-22',
 'STORIES_01': '2021-08-22',
 'WTA_SITE_NBS': '2021-08-22'}

cid, sbbolUserId, commonSegmentoUID, CTL_LOADING

distinct cid, sbbolUserId, commonSegmentoUID, не надо расскомментировать

In [59]:
class_cid_inn.scenarios_json[class_cid_inn.SCENARIO_ID]

In [ ]:
tmp_visit_part = self.load_visit_part()

        cid_sbbol, cid_segmento = self.get_cid_sbbol_and_segmento(tmp_visit_part)

        # Получение ИНН из связки с REPLICATIONGUID

        profile = self.load_user_profile()
        cid_sbbol_replicguid = cid_sbbol.join(profile, on=['sbbolUserId'], how='inner')


        customer = self.load_ma_customer()
        cid_sbbol_inn = cid_sbbol_replicguid.join(customer, on=['REPLICATIONGUID'], how='inner')


        # Получение ИНН из cookie_map

        #cookie_map = self.load_cookie_map()
        #active_cookie = self.get_active_cookie(cookie_map)

        #cid_segmento_inn = cid_segmento.join(active_cookie, 
        #                                   on=(cid_segmento.commonSegmentoUID == active_cookie.CUST_COOKIE), 
        #                                  how="left_outer")

        # Join

        cid_sbbol_inn_segmento = cid_sbbol_inn.join(cid_segmento, on=['cid'], how='full_outer').distinct() \
                                                #.withColumn('inn_',f.coalesce('CU_INN','CUST_INN')) \
                                                #.drop('CU_INN','CUST_INN')\
                                                #.withColumnRenamed("inn_", "CU_INN")


        unified_customer = self.load_unified_customer()

        cid_sbbol_inn_segmento_crm = cid_sbbol_inn_segmento.join(unified_customer, 
                                                                on=(cid_sbbol_inn_segmento.CU_INN == unified_customer.inn), how='leftouter') \
                                                                .dropDuplicates(['cu_inn','sbbolUserId', 'commonSegmentoUID', 'crm_id'])

        
        cid_sbbol_inn_segmento_crm_dt = self.add_load_dt(cid_sbbol_inn_segmento_crm)
        final_df = self.create_final_df(cid_sbbol_inn_segmento_crm_dt)